In [1]:
!pip install datasets

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   -- ------------------------------------- 1.3/25.8 MB 5.6 MB/s eta 0:00:05
   --- ------------------------------------ 2.1/25.8 MB 4.9 MB/s eta 0:00:05
   ---- ----------------------------------- 2.6/25.8 MB 4.0 MB/s eta 0:00:06
   ---- ----------------------------------- 3.1/25.8 MB 3.7 MB/s eta 0:00:07
   ----- ---------------------------------- 3.7/25.8 MB 3.6 MB/s eta 0:00:07
   ------ --------------------------------- 4.5/25.8 MB 3.5 MB/s eta 0:00:07
   ------- -------------------------------- 5.0/25.8 MB 3.4 MB/s eta 0:00:07
   -------- ------------------------------- 5.5/25.8 MB 3.3 MB/s eta 0:00:07
   ---------- ----------------------------- 6.6/25.8 MB 3.5 MB/s eta 0:00:06
   ------------- -------------------------- 8.4/25.8 MB 4.0 MB/s eta 0:00:05
   --------------- --------

In [2]:
from datasets import load_dataset
ds = load_dataset("roneneldan/TinyStories")

c:\Users\SATHVIK\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SATHVIK\anaconda3\envs\medibot\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SATHVIK\.cache\huggingface\hub\datasets--roneneldan--TinyStories. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to 

In [4]:
!pip install tiktoken
import tiktoken
import os
import numpy as np
from tqdm.auto import tqdm

enc = tiktoken.get_encoding("gpt2")

# Some functions from https://github.com/karpathy/nanoGPT/blob/master/data/openwebtext/prepare.py

def process(example):
    ids = enc.encode_ordinary(example['text']) # encode_ordinary ignores any special tokens
    out = {'ids': ids, 'len': len(ids)}
    return out

if not os.path.exists("train.bin"):
    tokenized = ds.map(
        process,
        remove_columns=['text'],
        desc="tokenizing the splits",
        num_proc=8,
        )
    # concatenate all the ids in each dataset into one large file we can use for training
    for split, dset in tokenized.items():
        arr_len = np.sum(dset['len'], dtype=np.uint64)
        filename = f'{split}.bin'
        dtype = np.uint16 # (can do since enc.max_token_value == 50256 is < 2**16)
        arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
        total_batches = 1024

        idx = 0
        for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'):
            # Batch together samples for faster write
            batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
            arr_batch = np.concatenate(batch['ids'])
            # Write into mmap
            arr[idx : idx + len(arr_batch)] = arr_batch
            idx += len(arr_batch)
        arr.flush()

tokenizing the splits (num_proc=8):   0%|          | 0/2119719 [00:02<?, ? examples/s]


NameError: name 'enc' is not defined